<H1><P style='text-align:center;'> Minesweeper Game</P></H1>

In [ ]:
#This cell contains the full code................................................................................................
import pygame
import sys

from random import randrange

# Colors used
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
GRAY = (127, 127, 127)
# Sets the WIDTH and HEIGHT of each grid location
WIDTH = 30
HEIGHT = 30
# Sets the starting number of squares
NSQUARES = 10
# Sets the margin between each cell
MARGIN = 5
MENU_SIZE = 40
LEFT_CLICK = 1
RIGHT_CLICK = 3


# Class that holds the game logic          
class Game:
    
    
    def __init__(self):
        # Create a grid of NSQUARES x NSQUARES
        self.grid = [[self.Cell(x, y) for x in range(NSQUARES)] for y in range(NSQUARES)]
        self.init = False
        self.game_lost = False
        self.game_won = False
        self.num_bombs = 10
        self.squares_x = NSQUARES
        self.squares_y = NSQUARES
        self.resize = False
        self.flag_count = 0

    def draw(self):
        # Set the screen background color
        screen.fill(BLACK)
        # Draw the grid
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                color = WHITE
                if self.grid[row][column].is_visible:
                     color = RED if self.grid[row][column].has_bomb else GRAY  
                elif self.grid[row][column].has_flag:
                    color = BLUE
                pygame.draw.rect(screen,
                                color,
                                [(MARGIN + WIDTH) * column + MARGIN,
                                (MARGIN + HEIGHT) * row + MARGIN + MENU_SIZE,
                                WIDTH,
                                HEIGHT])
                self.grid[row][column].show_text()
        
    # Adjusts the grid when the screen size has changed
    def adjust_grid(self, sizex, sizey):
        global screen
        self.squares_x = (sizex - MARGIN) // (WIDTH + MARGIN)
        self.squares_y = (sizey - MARGIN - MENU_SIZE) // (HEIGHT + MARGIN)
        if self.squares_x < 8:
            self.squares_x = 8
        if self.squares_y < 8:
            self.squares_y = 8
        if self.num_bombs > (self.squares_x * self.squares_y) // 3:
            self.num_bombs = self.squares_x * self.squares_y // 3
        self.grid = [[self.Cell(x, y) for x in range(self.squares_x)] for y in range(self.squares_y)]
        size = ((self.squares_x*(WIDTH + MARGIN) + MARGIN), (self.squares_y*(HEIGHT + MARGIN) + MARGIN + MENU_SIZE))
        screen = pygame.display.set_mode(size, pygame.RESIZABLE)

    # Makes all cells visible when user loses
    def game_over(self):
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                if self.grid[row][column].has_bomb:
                    self.grid[row][column].is_visible = True
                self.grid[row][column].has_flag = False

    # Changes the number of bombs placed and caps it
    def change_num_bombs(self, bombs):
        self.num_bombs += bombs
        if self.num_bombs < 1:
            self.num_bombs = 1
        elif self.num_bombs > (self.squares_x * self.squares_y) // 3:
            self.num_bombs = self.squares_x * self.squares_y // 3
        self.reset_game() 

    # Place BOMBS on random places
    def place_bombs(self, row, column):
        bombplaced = 0
        while bombplaced < self.num_bombs:
            x = randrange(self.squares_y)
            y = randrange(self.squares_x)
            if not self.grid[x][y].has_bomb and not (row == x and column == y):
                self.grid[x][y].has_bomb = True
                bombplaced += 1
        self.count_all_bombs()
        if self.grid[row][column].bomb_count != 0:
            self.reset_game()
            self.place_bombs(row, column)
        
    # Count all bombs next to a cell (3x3) for the entire grid
    def count_all_bombs(self):
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                self.grid[row][column].count_bombs(self.squares_y, self.squares_x)
    
    # Restarts the game
    def reset_game(self):
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                self.init = False
                self.grid[row][column].is_visible = False
                self.grid[row][column].has_bomb = False
                self.grid[row][column].bomb_count = 0
                self.grid[row][column].test = False
                self.grid[row][column].has_flag = False
                self.game_lost = False
                self.game_won = False
                self.flag_count = 0

    def check_victory(self):   
        count = 0
        total = self.squares_x * self.squares_y
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                if self.grid[row][column].is_visible:
                    count += 1
        if ((total - count) == self.num_bombs) and not self.game_lost:
            self.game_won = True
            for row in range(self.squares_y):
                for column in range(self.squares_x):
                    if self.grid[row][column].has_bomb:
                        self.grid[row][column].has_flag = True
        
    
    def count_flags(self):
        total_flags = 0
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                if self.grid[row][column].has_flag:
                            total_flags += 1
        self.flag_count = total_flags

    # Handle for grid clicks
    def click_handle(self, row, column, button):
        if button == LEFT_CLICK and self.game_won:
                self.reset_game()
        elif button == LEFT_CLICK and not self.grid[row][column].has_flag: 
            if not self.game_lost:
                # Place bombs after first click so you never click a bomb first
                if not self.init:
                    self.place_bombs(row, column)
                    self.init = True
                # Set the click square to visible
                self.grid[row][column].is_visible = True
                self.grid[row][column].has_flag = False
                if self.grid[row][column].has_bomb:
                    self.game_over()
                    self.game_lost = True
                if self.grid[row][column].bomb_count == 0 and not self.grid[row][column].has_bomb:
                    self.grid[row][column].open_neighbours(self.squares_y, self.squares_x)
                self.check_victory()
            else:
                self.game_lost = False
                self.reset_game()
        
        elif button == RIGHT_CLICK and not self.game_won:
            if not self.grid[row][column].has_flag:
                if self.flag_count < self.num_bombs and not self.grid[row][column].is_visible:
                    self.grid[row][column].has_flag = True
            else:
                self.grid[row][column].has_flag = False
            self.count_flags()


    # Game Sub-Class for each Cell of the grid
    class Cell:

        def __init__(self, x, y):
            self.x = x
            self.y = y
            self.is_visible = False
            self.has_bomb = False
            self.bomb_count = 0
            self.text = ""
            self.test = False
            self.has_flag = False

        # Handle for the number of bombs text
        def show_text(self):
            if self.is_visible:
                if self.bomb_count == 0:
                    self.text = font.render("", True, BLACK)
                else:
                    self.text = font.render(str(self.bomb_count), True, BLACK)
                screen.blit(self.text, (self.x * (WIDTH + MARGIN) + 12, self.y * (HEIGHT + MARGIN) + 10 + MENU_SIZE))
        
        # Counts how many bombs are next to this cell (3x3)
        def count_bombs(self, squaresx, squaresy):
            if not self.test:
                self.test = True
                if not self.has_bomb:
                    for column in range(self.x - 1 , self.x + 2):
                        for row in range(self.y - 1 , self.y + 2):
                            if (row >= 0 and row < squaresx and column >= 0 and column < squaresy
                                and not (column == self.x and row == self.y)
                                and game.grid[row][column].has_bomb):
                                    self.bomb_count += 1
        
        # Open all cells next to the clicked cell with zero bombs nearby
        def open_neighbours(self, squaresx, squaresy):
            column = self.x
            row = self.y
            for row_off in range(-1, 2):
                for column_off in range(-1, 2):
                    if ((row_off == 0 or column_off == 0) and row_off != column_off
                        and row+row_off >= 0 and column+column_off >=0 and row+row_off < squaresx and column+column_off < squaresy):
                            game.grid[row + row_off][column + column_off].count_bombs(game.squares_y, game.squares_x)
                            if not game.grid[row + row_off][column + column_off].is_visible and not game.grid[row + row_off][column + column_off].has_bomb:  
                                    game.grid[row + row_off][column + column_off].is_visible = True
                                    game.grid[row + row_off][column + column_off].has_flag = False
                                    if game.grid[row + row_off][column + column_off].bomb_count == 0: 
                                        game.grid[row + row_off][column + column_off].open_neighbours(game.squares_y, game.squares_x)


class Menu():

    def __init__(self):
        self.width = pygame.display.get_surface().get_width() - 2*MARGIN
        self.btn_minus = self.Button(10, 10, 20, 20, "-", 6, -3)
        self.btn_plus = self.Button(60, 10, 20, 20, "+", 3, -4)
        self.btn_flags = self.Button(280, 16, 10, 10, "")
        self.btn_flags.background = BLUE
        self.label_bombs = self.Label(30, 10)
        self.label_game_end = self.Label(100, 10)
        self.label_flags = self.Label(self.width - 50, 10)

    def click_handle(self, obj):
        if self.btn_minus.click_handle():
            obj.change_num_bombs(-1)
        if self.btn_plus.click_handle():
            obj.change_num_bombs(1)
        
    def draw(self, obj):
        self.width = pygame.display.get_surface().get_width() - 2*MARGIN 
        pygame.draw.rect(screen, GRAY, [MARGIN, 0, self.width, MENU_SIZE])
        self.btn_minus.draw(screen)
        self.btn_plus.draw(screen)
        self.btn_flags.draw(screen)
        self.label_bombs.show(screen, game.num_bombs)
        self.label_flags.show(screen, game.flag_count)
        if obj.game_lost:
            self.label_game_end.show(screen, "Game Over")
        elif obj.game_won:
            self.label_game_end.show(screen, "You Won!")
    

    class Label:
    
        def __init__(self, x, y):
            self.x = x
            self.y = y
            self.text = ""
        
        def show(self, surface, value): 
            text = str(value)
            self.text = font.render(text, True, BLACK)     
            surface.blit(self.text, (self.x, self.y))
    

    class Button:

        def __init__(self, x, y, width, height, text, xoff=0, yoff=0):
            self.x = x
            self.y = y
            self.height = height
            self.width = width
            self.background = WHITE
            self.text = text
            self.x_offset = xoff
            self.y_offset = yoff

        def draw(self, surface):
            pygame.draw.ellipse(surface, self.background, [self.x, self.y, self.width, self.height], 0)
            text = font.render(self.text, True, BLACK)     
            surface.blit(text, (self.x + self.x_offset, self.y + self.y_offset))
        
        def click_handle(self):
            pos = pygame.mouse.get_pos()
            if pos[0] > self.x and pos[1] > self.y and pos[0] < (self.x + self.width) and pos[1] < (self.y + self.height):
                return True
            else:
                return False


# Initialize pygame and sets screen size and caption
pygame.init()
size = (NSQUARES*(WIDTH + MARGIN) + MARGIN, (NSQUARES*(HEIGHT + MARGIN) + MARGIN) + MENU_SIZE)
screen = pygame.display.set_mode(size, pygame.RESIZABLE)
pygame.display.set_caption("Minesweeper")
# Font to use in the entire game
font = pygame.font.Font('freesansbold.ttf', 24)
# Create instances for Game and Menu
game = Game()
menu = Menu()
clock = pygame.time.Clock()
# Main loop
while True:
    for event in pygame.event.get():
        # Closes the game if user clicked the X
        if event.type == pygame.QUIT:  
            pygame.quit()
            sys.exit()
        # Mouse clicks event
        elif event.type == pygame.MOUSEBUTTONDOWN:
                # Get mouse position
                position = pygame.mouse.get_pos()
                # Change the screen coordinates to grid coordinates and caps max values
                column = position[0] // (WIDTH + MARGIN)
                row = (position[1] - MENU_SIZE) // (HEIGHT + MARGIN)
                if row >= game.squares_y:
                    row = game.squares_y - 1
                if column >= game.squares_x:
                    column = game.squares_x - 1
                if row >= 0:
                    game.click_handle(row, column, event.button)
                else:
                    menu.click_handle(game)
        # Event for screen resize    
        elif event.type == pygame.VIDEORESIZE:
            if game.resize: 
                game.adjust_grid(event.w, event.h)
                game.reset_game()
            else:  
                game.resize = True
    game.draw()
    menu.draw(game)
    clock.tick(10)
    # Update the screen
    pygame.display.flip()

pygame 2.5.1 (SDL 2.28.2, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
import pygame 
import sys

from random import randrange



Importing pygame gives us access to various classes and functions, allowing you to create interactive applications for example
* pygame.display.set_mode()  
* pygame.time.Clock()

Import the sys module in Python can help us gain access to various functions and attributes that allow you to interact with the Python interpreter and the system environment.
for example
* sys.exit()
* sys.platform

The random module provides various functions for generating random numbers. The randrange function, in particular, allows you to generate a random integer within a specified range


In [ ]:
# color tuples  are often used to represent colors using the RGB color model.
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
GRAY = (127, 127, 127)
#Sets the WIDTH and HEIGHT of each grid location
WIDTH = 30
HEIGHT = 30
#Sets the starting number of squares
NSQUARES = 10
#Sets the margin between each cell
MARGIN = 5
MENU_SIZE = 40
LEFT_CLICK = 1
RIGHT_CLICK = 3

In [ ]:
class Game:
    
    
    def __init__(self):
        self.grid = [[self.Cell(x, y) for x in range(NSQUARES)] for y in range(NSQUARES)]
        self.game_lost = False 
        self.game_won = False 
        self.num_bombs = 10 
        self.squares_x = NSQUARES 
        self.squares_y = NSQUARES 
        self.resize = False 
        self.flag_count = 0 

A **class** is a blueprint for creating objects. It defines a set of attributes (variables) and methods (functions) that the objects of that class will have.
so the above class defines a class named Game and its init method. The *init* method is a special method that is automatically called when an object of the class is created.

**self.init**: A boolean indicating whether the game has been initialized.

**self.game_lost**: A boolean indicating whether the game has been lost.

**self.game_won**: A boolean indicating whether the game has been won.

**self.num_bombs**: The number of bombs in the game.

**self.squares_x**: The number of squares in the x-direction of the grid.

**self.squares_y**: The number of squares in the y-direction of the grid.

**self.resize**: A boolean indicating whether the game window needs to be resized.

**self.flag_count**: The number of flags placed on the grid.

In [ ]:
def draw(self):
        screen.fill(BLACK)
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                color = WHITE
                if self.grid[row][column].is_visible:
                     color = RED if self.grid[row][column].has_bomb else GRAY  
                elif self.grid[row][column].has_flag:
                    color = BLUE
                pygame.draw.rect(screen,
                                color,
                                [(MARGIN + WIDTH) * column + MARGIN,
                                (MARGIN + HEIGHT) * row + MARGIN + MENU_SIZE,
                                WIDTH,
                                HEIGHT])
                self.grid[row][column].show_text()
        


This function contains code that iterates over each cell in the game grid, determines the color based on the cell's state, and then draws a rectangle representing the cell on the screen with the appropriate color.

**screen.fill(BLACK)**: This line fills the screen with a black color.

For the first two lines of *Iterations* self.squares_y  represent the number of rows  and self.squares_x variables represent the number of columns in the grid.
**color = WHITE**: the color is  initially set to white.

If the cell is *visible*, the color is set to *red* if the cell has a *bomb* , otherwise it is set to *gray* .
If the cell has a *flag*, the color is set to *blue* .


* **pygame.draw.rect()**:  is a function from the Pygame library used to draw rectangles.

* The first argument in this function(**screen**) is the surface object to draw on.
* The second argument is the color(**color**)of the rectangle.
* The third argument is a list specifying the position and size of the rectangle. It is calculated based on the current row and column, as well as the MARGIN, WIDTH, HEIGHT, and MENU_SIZE variables.

The next line calls the show_text() method of the current cell:"self.grid[row][column].show_text()"
This method is responsible for displaying the appropriate text

In [ ]:
def adjust_grid(self, sizex, sizey):
        global screen
        self.squares_x = (sizex - MARGIN) // (WIDTH + MARGIN)
        self.squares_y = (sizey - MARGIN - MENU_SIZE) // (HEIGHT + MARGIN)
        if self.squares_x < 8:
            self.squares_x = 8
        if self.squares_y < 8:
            self.squares_y = 8
        if self.num_bombs > (self.squares_x * self.squares_y) // 3:
            self.num_bombs = self.squares_x * self.squares_y // 3
        self.grid = [[self.Cell(x, y) for x in range(self.squares_x)] for y in range(self.squares_y)]
        size = ((self.squares_x*(WIDTH + MARGIN) + MARGIN), (self.squares_y*(HEIGHT + MARGIN) + MARGIN + MENU_SIZE))
        screen = pygame.display.set_mode(size, pygame.RESIZABLE)

##### **adjust_grid** is used to adjust the grid size in the game. The method takes two parameters, *sizex* and *sizey*, which represent the desired size of the grid.
Heres a breakdown of what the code does:
* It calculates the number of squares in the x and y directions based on the desired size of the grid (sizex and sizey).
* If the number of squares in either direction is less than 8, it sets it to 8. This ensures that the grid is at least 8x8.
* It checks if the number of bombs (self.num_bombs) is greater than one-third of the total number of squares in the grid. If it is, it reduces the number of bombs to one-third of the total number of squares.
* It creates a 2D list called grid using a nested list comprehension. Each element in the grid list is an instance of the Cell class.
* It calculates the size of the game window based on the number of squares in the grid and the dimensions of each square and the margin.
* It sets the screen variable to a new Pygame display surface with the calculated size, using the pygame.display.set_mode function.


In [ ]:
def game_over(self):
 # defines a method called game_over within a class.The self parameter refers to the instance of the class that the method is being called on
        for row in range(self.squares_y):
            for column in range(self.squares_x):
 # These two lines set up nested loops to iterate over each row and column in the grid. The range() function is used to generate a sequence of numbers from 0 up to, but not including, the specified value. self.squares_y and self.squares_x represent the number of rows and columns in the grid, respectively.             
                if self.grid[row][column].has_bomb:
                    self.grid[row][column].is_visible = True
 # This line checks if the current cell in the grid has a bomb (has_bomb attribute is True). If it does, it sets the is_visible attribute of that cell to True. This means that the bomb will be revealed and visible to the player.         
                    self.grid[row][column].has_flag = False
 #This line sets the has_flag attribute of the current cell to False. This means that if the cell had a flag placed on it, the flag will be removed.

In [ ]:

# Changes the number of bombs placed and caps it
def change_num_bombs(self, bombs):
        self.num_bombs += bombs
        if self.num_bombs < 1:
            self.num_bombs = 1
        elif self.num_bombs > (self.squares_x * self.squares_y) // 3:
            self.num_bombs = self.squares_x * self.squares_y // 3
        self.reset_game() 

##### This code uses a method that changes the number of bombs in a game. It ensures that the number of bombs is within a valid range and then resets the game to reflect the updated number of bombs
* self.num_bombs += bombs: adds the value of bombs to the num_bombs attribute of the object. It increases the number of bombs by the specified amount.
* if self.num_bombs < 1::checks if the num_bombs attribute is less than 1.
* self.num_bombs = 1: If the condition in the previous line is true, this line sets the num_bombs attribute to 1. This ensures that the number of bombs is always at least 1.
* elif self.num_bombs > (self.squares_x * self.squares_y) // 3::checks if the num_bombs attribute is greater than the maximum number of bombs allowed. The maximum number of bombs is calculated by dividing the total number of squares (squares_x * squares_y) by 3.
* self.num_bombs = self.squares_x * self.squares_y // 3: If the condition in the previous line is true, this line sets the num_bombs attribute to the maximum number of bombs allowed. This ensures that the number of bombs does not exceed the maximum limit.
* self.reset_game(): calls the reset_game() method.this method resets the game state, by repositioning the bombs and resetting other relevant variables.


In [ ]:
 # Random BOMB placement
def place_bombs(self, row, column):#This line defines a method called place_bombs that takes three parameters: self, row, and column. The self parameter is a reference to the instance of the class that this method belongs to.

        bombplaced =0 #This line initializes a variable bombplaced to 0. This variable will keep track of the number of bombs that have been placed.
        while bombplaced < self.num_bombs:
            x = randrange(self.squares_y)
            y = randrange(self.squares_x)
            if not self.grid[x][y].has_bomb and not (row == x and column == y):#This line checks if the randomly selected grid cell at coordinates (x, y) does not already have a bomb (self.grid[x][y].has_bomb) and is not the same cell as the one specified by the row and column parameters.
                self.grid[x][y].has_bomb = True
                bombplaced += 1
        self.count_all_bombs()
        #This block of code checks if the current cell (self.grid[row][column]) has a bomb count (bomb_count) that is not equal to 0. If this condition is true, it means that the current cell is not empty and has a bomb nearby. In this case, the game is reset.
        if self.grid[row][column].bomb_count != 0:
            self.reset_game()
            self.place_bombs(row, column)
        

The code is a function that randomly places bombs on a grid. It uses a while loop to place bombs until the desired number of bombs is reached. The x and y coordinates of each bomb are generated using the randrange function.
The condition ensures that a bomb is not placed on a square that already has a bomb or on the square where the player initially clicked.
After placing the bombs, the function calls self.count_all_bombs() to update the bomb count for each square. If the square where the player initially clicked has a non-zero bomb count, the game is reset and the bombs are placed again to ensure that the player doesn't lose on the first move.


In [ ]:
    # Count all bombs next to a cell (3x3) for the entire grid
    def count_all_bombs(self):
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                self.grid[row][column].count_bombs(self.squares_y, self.squares_x)

The **count_all_bombs()** method iterates over each cell in the game grid and calls the **count_bombs()** method on each cell to count the number of bombs in its neighboring cells. This is done for the entire grid.

In [ ]:
# Restart Game
def reset_game(self):
        for row in range(self.squares_y):
            for column in range(self.squares_x):
            
                self.init = False #represents whether the game has been initialized or not.
                self.grid[row][column].is_visible = False #represents whether the cell is visible or hidden.
                self.grid[row][column].has_bomb = False #represents whether the cell has a bomb or not.
                self.grid[row][column].bomb_count = 0 #represents the number of bombs adjacent to the cell.
                self.grid[row][column].test = False
                self.grid[row][column].has_flag = False # #represents whether the cell has a flag or not.
                self.game_lost = False #represents whether the game has been lost or not.
                self.game_won = False #represents whether the game has been won or not.
                self.flag_count = 0

In [ ]:
 def check_victory(self):   
        count = 0
        total = self.squares_x * self.squares_y
        for row in range(self.squares_y): #nested for loops that iterate over each row and column in the game grid
            for column in range(self.squares_x):
                if self.grid[row][column].is_visible:
                    count += 1
        if ((total - count) == self.num_bombs) and not self.game_lost:
            self.game_won = True
            for row in range(self.squares_y):
                for column in range(self.squares_x):
                    if self.grid[row][column].has_bomb:
                        self.grid[row][column].has_flag = True
  #checks if the difference between the total number of cells and the number of visible cells (count) is equal to the number of bombs (self.num_bombs). It also checks if the game has not been lost (not self.game_lost).                      

The **is_visible** attribute of each cell is checked, and if it is True, the count variable is incremented by 1. This counts the number of visible cells in the grid.

In [ ]:
def count_flags(self):
        total_flags = 0
        for row in range(self.squares_y):
            for column in range(self.squares_x):
                if self.grid[row][column].has_flag:
                            total_flags += 1
        self.flag_count = total_flags


The **count_flags()** method is used to count the number of cells in the game grid that have a flag placed on them. It initializes a variable called **total_flags** to 0. Then, it iterates over each cell in the grid using nested for loops. For each cell, it checks if the **has_flag** attribute is True, indicating that a flag has been placed on that cell. If the condition is true, it increments the total_flags variable by 1. Finally, it assigns the value of **total_flags** to the **flag_count** attribute of the class, which represents the total number of flags placed in the game. This method allows the game to keep track of the number of flags placed by the player.

In [ ]:
 # Handle for grid clicks
def click_handle(self, row, column, button): #handles the logic for handling left and right clicks on the game grid.
        if button == LEFT_CLICK and self.game_won:
                self.reset_game()
        elif button == LEFT_CLICK and not self.grid[row][column].has_flag: 
            if not self.game_lost:
                # Place bombs after first click so you never click a bomb first
                if not self.init:
                    self.place_bombs(row, column)
                    self.init = True
                # Set the click square to visible
                self.grid[row][column].is_visible = True
                self.grid[row][column].has_flag = False
                if self.grid[row][column].has_bomb:
                    self.game_over()
                    self.game_lost = True
                if self.grid[row][column].bomb_count == 0 and not self.grid[row][column].has_bomb:
                    self.grid[row][column].open_neighbours(self.squares_y, self.squares_x)
                self.check_victory()
            else:
                self.game_lost = False
                self.reset_game()
        
        elif button == RIGHT_CLICK and not self.game_won:
            if not self.grid[row][column].has_flag:
                if self.flag_count < self.num_bombs and not self.grid[row][column].is_visible:
                    self.grid[row][column].has_flag = True
            else:
                self.grid[row][column].has_flag = False
            self.count_flags()



1.Left Click:
If the button is a left click and the game has already been won , the game is reset.
If the button is a left click and the current cell does not have a flag , the following actions are performed:
* If the game has not been lost (not self.game_lost), the following actions are performed:
* If it is the first click (not self.init), bombs are placed on the grid using the place_bombs() method, and the init attribute is set to True.
* The clicked cell is set to visibleand does not have a flag (self.grid[row][column].has_flag = False).
* If the clicked cell has a bomb (self.grid[row][column].has_bomb), the game is over and  self.game_lost is set to True.
* If the clicked cell does not have a bomb and has a bomb count of 0 , the open_neighbours() method is called on the clicked cell to reveal its neighboring cells.
* The check_victory() method is called to check if the game has been won.
* If the game has been lost (self.game_lost is True), the game_lost flag is reset to False, and the game is reset by calling the reset_game() method.

2.Right Click:
If the button is a right click and the game has not been won, the following actions are performed:
If the current cell does not have a flag, the following conditions are checked:
* If the number of flags placed (self.flag_count) is less than the total number of bombs (self.num_bombs) and the current cell is not visible (not self.grid[row][column].is_visible), a flag is placed on the current cell (self.grid[row][column].has_flag = True).
* If the current cell already has a flag (self.grid[row][column].has_flag), the flag is removed by setting self.grid[row][column].has_flag to False.
* Finally, the count_flags() method is called to update the flag_count attribute.

In [ ]:
 # Game Sub-Class for each Cell of the grid
    class Cell:

        def __init__(self, x, y):
            self.x = x ##specify the position of the cell within the game grid.
            self.y = y #specify the position of the cell within the game grid.
            self.is_visible = False#determines whether the cell is currently visible to the player. Initially, all cells are set to be invisible.
            self.has_bomb = False #property indicates whether the cell contains a bomb. 
            #cells are initialized without bombs.
            self.bomb_count = 0 #property represents the number of neighboring cells that contain bombs. 
            self.text = ""
            self.test = False #used to display information about the cell, such as the bomb count or any other relevant text
            self.has_flag = False # indicates whether the cell has been flagged by the player.


The Cell class above represents a cell in Minesweeper. Each cell has properties such as its coordinates (x and y), visibility (is_visible), presence of a bomb (has_bomb), number of neighboring bombs (bomb_count), text representation (text), a test property (test), and whether it has a flag (has_flag).

In [ ]:
# Handle for the number of bombs text
        def show_text(self):#refers to the instance of the class that the method is being called on
            if self.is_visible:#checks if the is_visible attribute of the instance is True,so if it is true the code will proceed.if it is not it does nothing.  
                if self.bomb_count == 0: #It renders the string representation of self.bomb_count using the font object and assigns it to the text attribute of the instance.
                    self.text = font.render("", True, BLACK) #the BLACK color is used for rendering.
                else:
                    self.text = font.render(str(self.bomb_count), True, BLACK) # the text attribute is blitted (drawn) onto the screen at the position (self.x * (WIDTH + MARGIN) + 12, self.y * (HEIGHT + MARGIN) + 10 + MENU_SIZE). The screen object is used for blitting.
                screen.blit(self.text, (self.x * (WIDTH + MARGIN) + 12, self.y * (HEIGHT + MARGIN) + 10 + MENU_SIZE))
        

In [ ]:
# Counts how many bombs are next to this cell (3x3)
        def count_bombs(self, squaresx, squaresy):
            if not self.test: 
                self.test = True
                if not self.has_bomb:
                    for column in range(self.x - 1 , self.x + 2):
                        for row in range(self.y - 1 , self.y + 2):
                            if (row >= 0 and row < squaresx and column >= 0 and column < squaresy
                                and not (column == self.x and row == self.y)
                                and game.grid[row][column].has_bomb):
                                    self.bomb_count += 1
        

The above code uses a method called count_bombs that counts the number of bombs surrounding a given cell in a 3x3 grid. It seems to be a part of a larger program or class.
Here's a breakdown of how the code works:
* The method **count_bombs** takes two parameters: *squaresx* and *squaresy*, which likely represent the dimensions of the grid.

* The code checks if the variable **self.test** is False. If it is, it sets it to True. This suggests that the code is using self.test as a flag to ensure that the counting operation is performed only once.
* If the current cell (self) does not have a bomb (self.has_bomb is False), the code proceeds to count the number of bombs in the surrounding cells.
* It uses nested loops to iterate over the cells in a 3x3 grid centered around the current cell (self).
* For each cell in the grid, the code checks if the cell is within the boundaries of the grid (row >= 0 and row < squaresx and column >= 0 and column < squaresy).
* It also checks if the current cell is not the same as the center cell (column == self.x and row == self.y).
* If the above conditions are met and the cell being checked (game.grid[row][column]) has a bomb (has_bomb is True), it increments the bomb_count variable by 1.

In [ ]:
 # Open all cells next to the clicked cell with zero bombs nearby
        def open_neighbours(self, squaresx, squaresy):
            column = self.x
            row = self.y
            for row_off in range(-1, 2):
                for column_off in range(-1, 2):
                    if ((row_off == 0 or column_off == 0) and row_off != column_off
                        and row+row_off >= 0 and column+column_off >=0 and row+row_off < squaresx and column+column_off < squaresy):
                            game.grid[row + row_off][column + column_off].count_bombs(game.squares_y, game.squares_x)
                            if not game.grid[row + row_off][column + column_off].is_visible and not game.grid[row + row_off][column + column_off].has_bomb:  
                                    game.grid[row + row_off][column + column_off].is_visible = True
                                    game.grid[row + row_off][column + column_off].has_flag = False
                                    if game.grid[row + row_off][column + column_off].bomb_count == 0: 
                                        game.grid[row + row_off][column + column_off].open_neighbours(game.squares_y, game.squares_x)




The open_neighbours function is a method that belongs to a class. It takes two parameters, squaresx and squaresy, which represent the dimensions of a grid.

Inside the function, there are two nested loops that iterate over the neighboring cells of a given cell. The loops iterate over the values -1, 0, and 1, which represent the offsets from the current cell's position.
The condition inside the nested loops checks if the neighboring cell is within the boundaries of the grid. If it is, the function performs some operations on that cell.

First, it calls the count_bombs method on the neighboring cell, passing the dimensions of the grid as arguments. This method is not defined in the given code snippet, but it likely counts the number of bombs in the neighboring cell.

Next, it checks if the neighboring cell is not visible and does not have a bomb. If this condition is true, it sets the is_visible property of the cell to True, indicating that it is now visible. It also sets the has_flag property of the cell to False, indicating that it does not have a flag.

Finally, if the neighboring cell has a bomb count of 0, it recursively calls the open_neighbours method on that cell, passing the dimensions of the grid as arguments. This recursive call is used to open the neighboring cells of the current cell if they also have a bomb count of 0.

In [ ]:
class Menu():

    def __init__(self):
        self.width = pygame.display.get_surface().get_width() - 2*MARGIN #the width of the display surface minus twice the MARGIN value.
        self.btn_minus = self.Button(10, 10, 20, 20, "-", 6, -3)
        self.btn_plus = self.Button(60, 10, 20, 20, "+", 3, -4)
        self.btn_flags = self.Button(280, 16, 10, 10, "")
        self.btn_flags.background = BLUE
        self.label_bombs = self.Label(30, 10)
        self.label_game_end = self.Label(100, 10)
        self.label_flags = self.Label(self.width - 50, 10)

 

The btn_minus, btn_plus, and btn_flags attributes are instances of the Button class. Each button is initialized with specific coordinates, dimensions, text, and other properties.

The label_bombs, label_game_end, and label_flags attributes are instances of the Label class. Each label is initialized with specific coordinates

In [ ]:
   def click_handle(self, obj):#takes two parameters: self and obj.
        if self.btn_minus.click_handle():
            #checks if the click_handle method of the btn_minus button is triggered. If it returns True, it means the button was clicked.
            obj.change_num_bombs(-1) 
            # method of the obj object and passes -1 as an argument. This is likely intended to decrease the number of bombs.
        if self.btn_plus.click_handle():
            #is triggered by calling self.btn_plus.click_handle(). If this method returns True, it means that the button was clicked. 
            obj.change_num_bombs(1)
            #intended to increase the number of bombs.
        
   

In [ ]:
 def draw(self, obj):
        self.width = pygame.display.get_surface().get_width() - 2*MARGIN 
        pygame.draw.rect(screen, GRAY, [MARGIN, 0, self.width, MENU_SIZE])
        self.btn_minus.draw(screen)
        self.btn_plus.draw(screen)
        self.btn_flags.draw(screen)
        # draws a button on the screen using the draw method of the btn_flags object.
        self.label_bombs.show(screen, game.num_bombs)
        self.label_flags.show(screen, game.flag_count)
        if obj.game_lost:
            self.label_game_end.show(screen, "Game Over")
        elif obj.game_won:
            self.label_game_end.show(screen, "You Won!")
    

   

* **self.width** = pygame.display.get_surface().get_width() - 2*MARGIN: This line calculates the width of the game area by subtracting twice the MARGIN value from the total width of the display surface. The MARGIN is typically used to create a border around the game area.

* **pygame.draw.rect(screen, GRAY, [MARGIN, 0, self.width, MENU_SIZE])**: This line draws a rectangle on the screen surface using the GRAY color. The rectangle's position and size are specified by the list [MARGIN, 0, self.width, MENU_SIZE]. This rectangle is typically used to represent a menu or a header at the top of the game screen.

* **self.btn_minus.draw(screen)**: This line calls the draw method of the btn_minus object, which is responsible for drawing a button on the screen. The screen surface is passed as an argument to the draw method.
* **self.btn_plus.draw(screen)**: Similar to the previous line, this line calls the draw method of the btn_plus object to draw another button on the screen.
* **self.label_bombs.show(screen, game.num_bombs)**: This line calls the show method of the label_bombs object to display the number of bombs on the screen. The screen surface and the num_bombs attribute of the game object are passed as arguments to the show method.
* **self.label_flags.show(screen, game.flag_count)**: Similar to the previous line, this line calls the show method of the label_flags object to display the number of flags on the screen. The screen surface and the flag_count attribute of the game object are passed as arguments to the show method.
* if **obj.game_lost** : self.label_game_end.show(screen, "Game Over"): This line checks if the game_lost attribute of the obj object is True. If it is, it calls the show method of the label_game_end object to display the "Game Over" message on the screen. and  **elif obj.game_won**: self.label_game_end.show(screen, "You Won!"): This line checks if the game_won attribute of the obj object is True. If it is, it calls the show method of the label_game_end object to display the "You Won!" message on the screen.

In [ ]:
 class Label:
    
        def __init__(self, x, y):
            self.x = x
            self.y = y
            self.text = ""
        
        def show(self, surface, value): 
            text = str(value)
            self.text = font.render(text, True, BLACK)     
            surface.blit(self.text, (self.x, self.y))
    

    

This code defines a class called Label that represents a text label on a graphical surface. The Label class has three attributes: x, y, and text. The x and y attributes represent the coordinates of the label's position on the surface, and the text attribute stores the actual text value of the label.

The show method is used to display the label on a given surface. It takes two parameters: surface, which represents the graphical surface on which the label will be displayed, and value, which is the value that will be rendered as text on the label.

Inside the show method, the value parameter is converted to a string using the str function. Then, the text attribute is updated by rendering the value string using a font object called font. The rendered text is stored in the text attribute.

Finally, the rendered text is blitted (i.e., copied) onto the surface at the coordinates specified by the x and y attributes using the blit method of the surface object. The text is rendered in black color (BLACK) as specified by the font.render function.

In [ ]:
class Button:

        def __init__(self, x, y, width, height, text, xoff=0, yoff=0):
            self.x = x
            self.y = y
            self.height = height
            self.width = width
            self.background = WHITE
            self.text = text
            self.x_offset = xoff
            self.y_offset = yoff

        def draw(self, surface):
            pygame.draw.ellipse(surface, self.background, [self.x, self.y, self.width, self.height], 0)
            text = font.render(self.text, True, BLACK)     
            surface.blit(text, (self.x + self.x_offset, self.y + self.y_offset))
        
        def click_handle(self):
            pos = pygame.mouse.get_pos()
            if pos[0] > self.x and pos[1] > self.y and pos[0] < (self.x + self.width) and pos[1] < (self.y + self.height):
                return True
            else:
                return False




This code defines a class with several methods for creating and handling a graphical button in a Pygame application. The __init__ method initializes the button's position, size, background color, and text. The draw method is responsible for rendering the button on the screen, using the Pygame library to draw an ellipse shape and display the text. The click_handle method checks if the button has been clicked by comparing the mouse position with the button's position and size.

This code provides a way to create and interact with a graphical button in a Pygame application. The button can be drawn on the screen, and its click status can be checked to perform specific actions when it is clicked.


In [ ]:
# Initialize pygame and sets screen size and caption
pygame.init()
size = (NSQUARES*(WIDTH + MARGIN) + MARGIN, (NSQUARES*(HEIGHT + MARGIN) + MARGIN) + MENU_SIZE)
screen = pygame.display.set_mode(size, pygame.RESIZABLE)
pygame.display.set_caption("Minesweeper")
# Font to use in the entire game
font = pygame.font.Font('freesansbold.ttf', 24)
# Create instances for Game and Menu
game = Game()
menu = Menu()
clock = pygame.time.Clock()
# Main loop
while True:
    for event in pygame.event.get():
        # Closes the game if user clicked the X
        if event.type == pygame.QUIT:  
            pygame.quit()
            sys.exit()
        # Mouse clicks event
        elif event.type == pygame.MOUSEBUTTONDOWN:
                # Get mouse position
                position = pygame.mouse.get_pos()
                # Change the screen coordinates to grid coordinates and caps max values
                column = position[0] // (WIDTH + MARGIN)
                row = (position[1] - MENU_SIZE) // (HEIGHT + MARGIN)
                if row >= game.squares_y:
                    row = game.squares_y - 1
                if column >= game.squares_x:
                    column = game.squares_x - 1
                if row >= 0:
                    game.click_handle(row, column, event.button)
                else:
                    menu.click_handle(game)
        # Event for screen resize    
        elif event.type == pygame.VIDEORESIZE:
            if game.resize: 
                game.adjust_grid(event.w, event.h)
                game.reset_game()
            else:  
                game.resize = True
    game.draw()
    menu.draw(game)
    clock.tick(10)
    pygame.display.flip()

This code is an implementation of the game Minesweeper using the Pygame library. It starts by initializing Pygame and setting the size of the game window based on the number of squares, their width and height, and the margin between them. The window is also given a title.\

A font is loaded to be used throughout the game for displaying text. Instances of the Game and Menu classes are created. The main loop begins, where events are continuously checked. If the user clicks the X button, the game window is closed. If the user clicks the mouse, the position of the click is obtained and converted to grid coordinates. The row and column values are capped to the maximum number of squares in each direction. The click is then handled by the Game or Menu class depending on the position.

If the user resizes the window, an event of type pygame.VIDEORESIZE is triggered. If the game's resize flag is set, the grid is adjusted to fit the new window size, and the game is reset. Otherwise, the resize flag is set to True to allow for resizing in the next event.
In each iteration of the main loop, the game and menu are drawn on the screen, and the clock is ticked to control the frame rate. Finally, the screen is updated to reflect the changes made in the loop.